<a href="https://colab.research.google.com/github/Resource-Efficiency-Collective/cement-theoretical-minimum/blob/main/Cement_Theoretical_Minimum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manufacturing of Clinker: Theoretical and Practical Minimum Energy Requirements

This notebook is supplementary information for the model described in the article:

**DOI:** https://doi.org/

This notebook is split into 3 tasks:

1.   Calculate theoretical minimum for clinker
2.   Calculate the clinker strength
3.   Calculate the compsition from the clinker phases using Bogue's equation

Run the first two cells setup the notebook.

For the cement strength, D28A accounts for the specific surface area for clinker, which was assumed as 350 m<sup>2</sup>/kg. D28 does not account the clinker specific surface area.

In [ ]:
# @title Press to install and load packages (can take up to 40 seconds)
%%capture
!pip install thermopy
!pip install hide_warnings


import gdown, os
from google.colab import files

# Import and unzip files -> You can then view them in the left files panel
folder, zip_path = 'example_data', 'example_data.zip'
gdown.download('https://drive.google.com/uc?id=1rit2Jhem-1PC5dyMl_1DQwgpJ7egf0dX', zip_path, quiet=True)
!unzip /content/example_data.zip

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from thermopy_modified.thermopy import nasa9polynomials as nasa9
from scipy.integrate import quad
from hide_warnings import hide_warnings
from IPython.display import clear_output, display


In [ ]:
# @title Press to load the models

@hide_warnings
def heat_embodied(Ta, Th, mat):
    #Import database
    db = nasa9.Database()

    # Declare materials
    alumina = db.set_compound('Al2O3')
    aluminium_silicate = db.set_compound('Al2SiO5(an)')  # anhydride
    calcite = db.set_compound('CaCO3(cr)')
    calcium_oxide = db.set_compound('CaO(cr)')
    carbon_dioxide = db.set_compound('CO2')
    carbon_gas = db.set_compound('C')  # if want graphite need to change to C(gr)
    carbon = db.set_compound('C(gr)')
    iron_oxide = db.set_compound('Fe2O3(cr)')
    magnesium_oxide = db.set_compound('MgO')
    magnesium_carbonate = db.set_compound('MgCO3(cr)')
    magnesium_silicate = db.set_compound('Mg2SiO4(cr)')
    potassium = db.set_compound('K2O')
    sodium = db.set_compound('Na2O')
    sulphur = db.set_compound('SO3')
    silica = db.set_compound('SiO2')
    water = db.set_compound('H2O(l)')
    water_vapour = db.set_compound('H2O')

    # Ta is the Low temperature in celsius
    # Th  is the High temperature in celsius
    # mat is the material

    Heat = quad(eval(mat).heat_capacity, Ta + 273.16, Th + 273.16,limit=5,)[0] / eval(mat).molecular_weight / 1000  # kJ/kg

    return Heat

def rate(CaO,
         SiO2,
         Al2O3,
         Fe2O3,
         MgO,
         K2O,
         Na2O,
         SO3,
         water_raw,loss_clay):

    # Correct loss_clay
    loss_clay = loss_clay/100  # 12

    # %% Import database
    db = nasa9.Database()

    # %% Declare materials
    alumina = db.set_compound('Al2O3')
    calcite = db.set_compound('CaCO3(cr)')
    calcium_oxide = db.set_compound('CaO(cr)')
    carbon_dioxide = db.set_compound('CO2')
    iron_oxide = db.set_compound('Fe2O3(cr)')
    magnesium_oxide = db.set_compound('MgO')
    magnesium_silicate = db.set_compound('Mg2SiO4(cr)')
    potassium = db.set_compound('K2O')
    sodium = db.set_compound('Na2O')
    sulphur = db.set_compound('SO3')
    silica = db.set_compound('SiO2')
    water = db.set_compound('H2O(l)')
    water_vapour = db.set_compound('H2O')

    # Other variables
    # Molecular weight
    alumina_mw=alumina.molecular_weight
    calcite_mw = calcite.molecular_weight
    calcium_oxide_mw = calcium_oxide.molecular_weight
    carbon_dioxide_mw = carbon_dioxide.molecular_weight
    iron_oxide_mw = iron_oxide.molecular_weight
    magnesium_oxide_mw = magnesium_oxide.molecular_weight
    magnesium_silicate_mw = magnesium_silicate.molecular_weight
    potassium_mw = potassium.molecular_weight
    sodium_mw = sodium.molecular_weight
    sulphur_mw = sulphur.molecular_weight
    silica_mw = silica.molecular_weight
    water_mw = water.molecular_weight
    water_vapour_mw = water_vapour.molecular_weight

    # Declare share of clays and shales and limestone
    # Ignore losses due to water content
    share_limestone = CaO / 100  # Assumes content of calcite coming exclusively from limestone
    share_clay = 1 - share_limestone
    loss_limestone = 0.44  # 37

    # The dry raw meal that will enter the kiln
    raw_meal_mass = 1 / (share_limestone * (1 - loss_limestone) + share_clay * (1 - loss_clay))

    # Mass of limestone dry
    mass_limestone = raw_meal_mass * share_limestone
    # Mass of clays and shales dry
    mass_clay = raw_meal_mass * share_clay

    # Mass of raw meal wet
    raw_meal_mass_wet = raw_meal_mass * (1 + water_raw / 100)

    # Organize the shares in a vector
    raw_meal_share = np.array([SiO2, Al2O3, Fe2O3, CaO, MgO, K2O, Na2O, SO3, water_raw])

    # Normalise with the water content
    raw_meal_share = raw_meal_share / sum(raw_meal_share) * 100

    # Create vector to track mass and energy change
    mass_raw = raw_meal_mass_wet
    heat_raw = np.array([])
    energy_raw = np.array([])
    # Track change of temperature
    T_stage = []

    # %% Define heat capacity of the raw meal

    # Molar mass
    raw_meal_materials = ['silica', 'alumina', 'iron_oxide', 'calcium_oxide', 'magnesium_oxide',
                          'potassium', 'sodium', 'sulphur', 'water']

    # create a dictionary for the raw meal
    dicts_raw = dict(zip(raw_meal_materials, raw_meal_share))

    ### Drying stage
    # define initial temperature
    T_dry_amb = 25

    # define final temperature
    T_dry_final = 105
    # update temperature
    T_stage = np.append(T_stage, [T_dry_amb, T_dry_final])
    # initial step assumes that  the raw meal will be heated up to 100 celsius
    H = []

    for i in dicts_raw.keys():
        H = np.append(H, heat_embodied(T_dry_amb, 100, i))

    H_100 = H * raw_meal_share / 100  # kJ/kg

    # extra energy to vaporize water
    heat_vaporization_water = 2260  # kJ/kg
    H_Water_vap = raw_meal_share[
                      -1] / 100 * heat_vaporization_water  # kJ /kg ready to be multiplied by the mass of raw meal


    # further heating after the water vapour transition >100 celsius
    dicts_raw['water_vapour'] = dicts_raw.pop('water')

    H = []

    for i in dicts_raw.keys():
        H = np.append(H, heat_embodied(100, T_dry_final, i))

    H_105 = H * raw_meal_share / 100  # kJ/kg

    total_heat_drying = sum(H_100) + H_Water_vap + sum(H_105)

    # update mass
    mass_raw = np.append(mass_raw, raw_meal_mass)
    mass_lost = [0, raw_meal_mass_wet - raw_meal_mass]

    # update energy
    heat_raw = np.append(heat_raw, total_heat_drying)
    energy_raw = np.append(energy_raw, + 0)  # heat of vaporization not here

    # %% Next stage there is no water
    ### Clay decomposition 400 - 650 celsius

    # Temperature assumed for clay decomposition
    T_clay = 525
    T_stage = np.append(T_stage, T_clay)

    raw_meal_share = raw_meal_share[:-1] * (100 / sum(raw_meal_share[:-1]))  # already normalized for the new mass

    # Update dictionary
    dicts_raw = dict(zip(raw_meal_materials[:-1], raw_meal_share))

    H_clay = []

    for i in dicts_raw.keys():
        H_clay = np.append(H_clay, heat_embodied(T_dry_final, T_clay, i))

    H_clay_heat = H_clay * raw_meal_share / 100  # kJ/kg

    # kaolinite 0.258 kg/mol
    # 113 kj/mol from cement kiln
    energy_clay_decomposition = 113 / 0.258  # kJ/kg
    H_clay_decomposition = mass_clay / raw_meal_mass * energy_clay_decomposition

    # update energy
    heat_raw = np.append(heat_raw, heat_raw[-1] + sum(H_clay_heat))
    energy_raw = np.append(energy_raw, energy_raw[-1] + H_clay_decomposition)

    mass_clay_lost = raw_meal_mass * share_clay * loss_clay
    mass_clay = mass_clay - mass_clay_lost

    # Update raw meal mass
    raw_meal_mass = raw_meal_mass - mass_clay_lost

    mass_raw = np.append(mass_raw, raw_meal_mass)
    # Keep track of the mass lost

    mass_lost = np.append(mass_lost, mass_lost[-1] + mass_clay_lost)


    # Exhaust gases at 525
    # Need to define the share between water vapour and CO2
    # Clays decomposition mainly water vapour


    #### Silica reaction
    # Temperature in range of 650 to 900
    # 2 CaCO3 + SiO2 → Ca2SiO4 + 2 CO2  +210.99 kJ/mol.
    # energy to heat
    T_silica = 775
    T_stage = np.append(T_stage, T_silica)
    H_si = []

    for i in dicts_raw.keys():
        H_si = np.append(H_si, heat_embodied(T_clay, T_silica, i))

    H_silica_heat = H_si * raw_meal_share / 100  # kJ/kg

    mol_silica = dicts_raw['silica'] / 100 / silica_mw
    mol_magnesium = dicts_raw['magnesium_oxide'] / 100 / magnesium_oxide_mw

    # 2 MgO + SiO2 → Mg2SiO4

    # The enthalpy change for this reaction at 25°C is -63.6 kJ/mol: at 775°C it is -65.4 kJ/mol.
    energy_magnesium = -65.4 / magnesium_silicate_mw
    H_magnesium = energy_magnesium * dicts_raw['magnesium_oxide'] / 100

    # here is the total mass of limestone that is consumed
    mass_limestone_consumed_silica = 2 * ((mol_silica - mol_magnesium / 2) * calcite_mw)

    energy_silica = 210.99 / silica_mw # kJ/kg
    H_silica = dicts_raw['silica'] / 100 * energy_silica

    # update energy
    heat_raw = np.append(heat_raw, heat_raw[-1] + sum(H_silica_heat))
    energy_raw = np.append(energy_raw, energy_raw[-1] + H_magnesium + H_silica)

    mass_CO2_silica = (mol_silica - mol_magnesium / 2) * 2 * carbon_dioxide_mw
    mass_limestone = mass_limestone - mass_limestone_consumed_silica
    raw_meal_mass = raw_meal_mass - mass_CO2_silica
    mass_raw = np.append(mass_raw, raw_meal_mass)
    mass_lost = np.append(mass_lost, mass_lost[-1] + mass_CO2_silica)

    ### Alumina and Iron oxide reaction
    # Temperature is in between 700 and 900 Celsius
    T_oxide = 800
    T_stage = np.append(T_stage, T_oxide)
    H_oxide = []

    for i in dicts_raw.keys():
        H_oxide = np.append(H_oxide, heat_embodied(T_silica, T_oxide, i))

    H_oxide_heat = H_oxide * raw_meal_share / 100  # kJ/kg


    mol_iron = dicts_raw['iron_oxide'] / 100 / iron_oxide_mw
    mol_alumina = dicts_raw['alumina'] / 100 / alumina_mw
    mass_iron_alumina = dicts_raw['alumina'] / 100 * (mol_iron / mol_alumina)
    mass_alumina = dicts_raw['alumina'] / 100 * (1 - mol_iron / mol_alumina)

    ### Alumina and Iron
    # 4 CaCO3 + Al2O3 + Fe2O3 → Ca4Al2Fe2O10 + 4 CO2
    # The enthalpy change for this reaction at 25°C is +661.9 kJ/mol: at 800°C it is +611.8 kJ/mol.
    # +631.6 kJ/mol  0.102 kg/mol
    energy_iron_alumina = 611.8 / 0.102  # kJ/kg
    H_iron_alumina = mass_iron_alumina * energy_iron_alumina  # kJ
    # 1 mol of Al2O3 generates 4 mol CO2
    mass_CO2_iron_alumina = mass_iron_alumina / alumina_mw* 4 * carbon_dioxide_mw

    # Alumina that does not react
    # 12 CaCO3 + 7 Al2O3 → Ca12Al14O33 + 12 CO2
    # The enthalpy change for this reaction at 25°C is +2060 kJ/mol: at 800°C it is +1892 kJ/mol.
    energy_alumina = 1892 / (7 * 0.102)  # kJ / kg
    H_alumina = mass_alumina * energy_alumina

    # update energy
    heat_raw = np.append(heat_raw, heat_raw[-1] + sum(H_oxide_heat))
    energy_raw = np.append(energy_raw, energy_raw[-1] + H_alumina + H_iron_alumina)

    mass_CO2_alumina = mass_alumina / alumina_mw * (12 / 7) * carbon_dioxide_mw

    mass_limestone_lost_oxide = mass_CO2_alumina + mass_CO2_iron_alumina
    mass_limestone = mass_limestone - mass_limestone_lost_oxide * (
            calcite_mw / carbon_dioxide_mw)

    # Update raw meal mass
    raw_meal_mass = raw_meal_mass - mass_limestone_lost_oxide
    mass_raw = np.append(mass_raw, raw_meal_mass)
    mass_lost = np.append(mass_lost, mass_lost[-1] + mass_limestone_lost_oxide)

    #### Calcination
    # Temperature in range of 900 to 1050
    T_calcination = 975
    T_stage = np.append(T_stage, T_calcination)
    H_calcination = []

    for i in dicts_raw.keys():
        H_calcination = np.append(H_calcination, heat_embodied(T_oxide, T_calcination, i))

    H_calcination_heat = H_calcination * raw_meal_share / 100  # kJ/kg

    # at 975°C it is +166.17 kJ/mol.
    energy_calcination = 166.7 / 0.100  # kJ/kg
    H_calcite = energy_calcination * mass_limestone / raw_meal_mass  # kJ

    # update energy
    heat_raw = np.append(heat_raw, heat_raw[-1] + sum(H_calcination_heat))
    energy_raw = np.append(energy_raw, energy_raw[-1] + H_calcite)

    mass_CO2_calcite = mass_limestone * (carbon_dioxide_mw / calcite_mw)

    # Here is mass of calcium oxide already, keeping nomenclature
    mass_limestone = mass_limestone - mass_CO2_calcite

    # Update raw meal mass
    raw_meal_mass = raw_meal_mass - mass_CO2_calcite
    mass_raw = np.append(mass_raw, raw_meal_mass)
    mass_lost = np.append(mass_lost, mass_lost[-1] + mass_CO2_calcite)


    ### Sintering
    # Temperature in range of 1300 to 1425
    # Ca2SiO4 + CaO → Ca3SiO5
    # +24.2 kJ/mol
    T_sintering = 1365
    T_stage = np.append(T_stage, T_sintering)
    H_sintering = []

    for i in dicts_raw.keys():
        H_sintering = np.append(H_sintering, heat_embodied(T_calcination, T_sintering, i))

    H_sintering_heat = H_sintering * raw_meal_share / 100  # kJ/kg

    energy_sintering = 24.2 * calcium_oxide_mw
    # this mass of limestone is CaO only kept the same variable name
    H_sinter = energy_sintering * mass_limestone / raw_meal_mass


    # update energy
    heat_raw = np.append(heat_raw, heat_raw[-1] + sum(H_sintering_heat))
    energy_raw = np.append(energy_raw, energy_raw[-1] + H_sinter)

    ### Figures
    # Figure parameters
    plt.style.use(['seaborn-v0_8-whitegrid'])
    plt.rcParams['font.family'] = 'sans-serif'  # sans-serif
    plt.rcParams['font.size'] = 14

    heat_raw = np.append(0, heat_raw)
    energy_raw = np.append(0, energy_raw)
    ####
    # Create a figure and subplots
    fig = plt.figure(figsize=(11, 6))


    # First subplot
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(T_stage, np.append(mass_raw, mass_raw[-1]), color='black', linestyle='-', linewidth=1.5, label='Meal')
    ax1.set_ylabel('Raw meal (kg)', weight='bold')
    ax1.set_title('Material Yield {:.2f} (%)'.format((mass_raw[-1] / mass_raw[0]) * 100), weight='bold')
    ax2 = ax1.twinx()
    ax2.plot(T_stage, np.append(mass_lost, mass_lost[-1]), color='red', linestyle='-', linewidth=1.5, label='Losses')
    ax2.set_ylabel('Mass Loss (kg)', color='k', weight='bold')
    ax1.grid(color='grey', linestyle='-.', linewidth=0.5, alpha=0.2)
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='center left')

    # Second subplot
    ax3 = fig.add_subplot(1, 2, 2)
    ax3.plot(T_stage, (heat_raw + energy_raw) / 1000, color='black', linestyle='-', linewidth=1.5, label='Total')
    ax3.plot(T_stage, heat_raw / 1000, color='orange', linestyle='-', linewidth=1.5, label='Heating')
    ax3.plot(T_stage, energy_raw / 1000, color='red', linestyle='-', linewidth=1.5, label='Reactions')
    ax3.set_ylabel('Energy required (MJ/kg)', weight='bold')
    ax3.set_xlabel('Temperature ($^o$C)', weight='bold')
    ax3.set_title('Energy intensity {:.2f} (MJ/kg)'.format((heat_raw[-1] + energy_raw[-1]) / 1000), weight='bold')
    ax3.grid(color='grey', linestyle='-.', linewidth=0.5, alpha=0.2)
    lines, labels = ax3.get_legend_handles_labels()
    ax3.legend(lines, labels, loc="upper left", frameon=False)

    # Adjust layout
    plt.tight_layout()

    # Show the figure
    plt.show()


    ##### Here  Starts Strength
    Bogue = np.array([[4.0710, -7.6024, -1.4297, -6.7187],
              [-3.0710, 8.6024, 1.0785, 5.0683],
              [0, 0, -1.6920, 2.6504],
              [0, 0, 3.0432, 0],
              ])


    C=[CaO,SiO2,Fe2O3,Al2O3]
    C=np.dot(Bogue,C)
    C3S=C[0]
    C2S=C[1]
    C3A=C[2]
    C4AF=C[3]
    Sum=sum(C)
    # Correct if below 100
    if Sum<100:
        print("\033[1mCurrent shares of clinker phases were corrected to match 100%\033[0m")
        CC3S = C3S*100/Sum
        C2S = C2S*100/Sum
        C3A = C3A*100/Sum
        C4AF = C4AF*100/Sum


    FCH = C3S + C2S + C3A - C4AF
    D28 = 0.7603 * FCH - 16.95  # MPa
    SSA =350 # kg/m2
    D28A = 0.4714 * FCH + 0.04231 * SSA - 12.7 # MPa

    # Data
    labels = [1.2,1.5]
    values = [D28,D28A]

    # Create the horizontal bar chart
    plt.figure(figsize=(6, 6))
    bars = plt.barh(labels, values, height=0.2, color=['#878787', '#4d4d4d'])

    # Add value labels at the end of each bar
    for bar, value in zip(bars, values):

        formatted_value = f"{value:.2f}"
        plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, formatted_value, va='center', ha='left',weight='bold')

    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().xaxis.set_ticks_position('none')
    plt.gca().yaxis.set_ticks_position('none')
    plt.gca().xaxis.set_visible(False)
    plt.yticks([1.2,1.5],['D28','D28A'],weight='bold')
    plt.title('Clinker strength (MPa)')
    plt.rcParams['axes.facecolor'] = 'white'
    plt.grid(c='none')
    plt.show()


def strength(a,b,c,d,e):
    C3S = a
    C2S = b
    C3A = c
    C4AF = d
    Sum=a+b+c+d
    # Correct if below 100
    if Sum<100:
        print("\033[1mCurrent ratios are unbalanced!Corrected to match 100%!\033[0m")
        C3S = a*100/Sum
        C2S = b*100/Sum
        C3A = c*100/Sum
        C4AF = d*100/Sum

    # Print if unbalanced
    if Sum>100:
        print("\033[1mCurrent ratios are unbalanced!Greater than 100%!\033[0m")
        C3S = a*100/Sum
        C2S = b*100/Sum
        C3A = c*100/Sum
        C4AF = d*100/Sum

    SSA = e # kg/m2

    FCH = C3S + C2S + C3A - C4AF
    D28 = 0.7603 * FCH - 16.95  # MPa
    D28A = 0.4714 * FCH + 0.04231 * SSA - 12.7 # MPa

    # Sample data
    labels = [1.2,1.5]
    values = [D28,D28A]
    # Figure parameters
    plt.style.use(['seaborn-v0_8-whitegrid'])
    plt.rcParams['font.family'] = 'sans-serif'  # sans-serif
    plt.rcParams['font.size'] = 14

    # Create the horizontal bar chart
    plt.figure(figsize=(8, 6))
    bars = plt.barh(labels, values, height=0.2, color=['#878787', '#4d4d4d'])

    # Add value labels at the end of each bar
    for bar, value in zip(bars, values):

        formatted_value = f"{value:.2f}"
        plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, formatted_value, va='center', ha='left',weight='bold')

    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().xaxis.set_ticks_position('none')
    plt.gca().yaxis.set_ticks_position('none')
    plt.gca().xaxis.set_visible(False)
    plt.yticks([1.2,1.5],['D28','D28A'],weight='bold')
    plt.title('Clinker strength (MPa)')
    plt.rcParams['axes.facecolor'] = 'white'
    plt.grid(c='none')
    plt.show()


def composition(a,b,c,d):
    C3S = a
    C2S = b
    C3A = c
    C4AF = d
    Sum=a+b+c+d

    # Print
    if Sum>100:
        print("\033[1mCurrent ratios are unbalanced!Greater than 100%!\033[0m")
        C3S = a*100/Sum
        C2S = b*100/Sum
        C3A = c*100/Sum
        C4AF = d*100/Sum

    Bogue = np.array([[4.0710, -7.6024, -1.4297, -6.7187],
              [-3.0710, 8.6024, 1.0785, 5.0683],
              [0, 0, -1.6920, 2.6504],
              [0, 0, 3.0432, 0],
              ])

    # Clinker composition
    Phases = np.array([C3S,C2S,C3A,C4AF])
    x = np.linalg.solve(Bogue, Phases)
    x = np.reshape(x, (1,4))

    df_raw = pd.DataFrame(x, columns=['CaO', 'SiO2', 'Fe2O3', 'Al2O3'])

    M = []
    S = []
    K = []
    Na = []
    SO = []
    total_2_axis = np.sum(df_raw, axis=1)

    for i in range(len(df_raw)):
        S = 100 - total_2_axis[i]  # np.sum(df_raw.iloc[i, :])
        M = np.append(M, 0.5 * S)
        K = np.append(K, 0.125 * S)
        Na = np.append(Na, 0.125 * S)
        SO = np.append(SO, 0.25 * S)

    df_raw['MgO'] = M
    df_raw['K2O'] = K
    df_raw['Na2O'] = Na
    df_raw['SO3'] = SO

    df_raw = df_raw.applymap(lambda x: "{:.2f}".format(x))
    df_raw = df_raw.replace('-0.00', 0)
    print("\033[1m Share of each component (%):\033[0m")
    print(df_raw.to_string(index=False))
    print()

In [ ]:
from ipywidgets.widgets.widget_selection import DescriptionStyle
# @title Press to run the models (can take up to 30 seconds)

CaO=widgets.FloatSlider(value=67.4, min=40.0, max=90.0, step=0.1, description='CaO (%):',style={'description_width': '64pt'})
SiO2=widgets.FloatSlider(value=25.5, min=10.0, max=40.0, step=0.1, description='SiO2 (%):',style={'description_width': '64pt'})
Al2O3=widgets.FloatSlider(value=3.1, min=1.0, max=30.0, step=0.1, description='Al2O3 (%):',style={'description_width': '64pt'})
Fe2O3=widgets.FloatSlider(value=2.0, min=1.0, max=20.0, step=0.1, description='Fe2O3 (%):',style={'description_width': '64pt'})
MgO=widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description='MgO (%):',style={'description_width': '64pt'})
K2O=widgets.FloatSlider(value=.25, min=.0, max=5.0, step=0.1, description='K2O (%):',style={'description_width': '64pt'})
Na2O=widgets.FloatSlider(value=.25, min=.0, max=5.0, step=0.1, description='Na2O (%):',style={'description_width': '64pt'})
SO3=widgets.FloatSlider(value=.5, min=.0, max=5.0, step=0.1, description='SO3 (%):',style={'description_width': '64pt'})
water_raw=widgets.FloatSlider(value=0.0, min=0.0, max=20.0, step=0.1, description='Humidity (%):',style={'description_width': '64pt'})
loss_clay=widgets.FloatSlider(value=6.0, min=0.0, max=10.0, step=0.1, description='Clay Loss (%):',style={'description_width': '64pt'})
button = widgets.Button(description='Run')


def plot_callback(button):
    # Clear the output area before plotting
    with output:
        clear_output(wait=False)
        print("Running the model... Can take up to 30 seconds...")
        rate(CaO.value,
         SiO2.value,
         Al2O3.value,
         Fe2O3.value,
         MgO.value,
         K2O.value,
         Na2O.value,
         SO3.value,
         water_raw.value,loss_clay.value)

# Output area
output = widgets.Output()
# Attach the callback function to the button click event
button.on_click(plot_callback)

# Display widgets and output area
display(CaO,
         SiO2,
         Al2O3,
         Fe2O3,
         MgO,
         K2O,
         Na2O,
         SO3,
         water_raw,loss_clay, button, output)

FloatSlider(value=67.4, description='CaO (%):', max=90.0, min=40.0, style=SliderStyle(description_width='64pt'…

FloatSlider(value=25.5, description='SiO2 (%):', max=40.0, min=10.0, style=SliderStyle(description_width='64pt…

FloatSlider(value=3.1, description='Al2O3 (%):', max=30.0, min=1.0, style=SliderStyle(description_width='64pt'…

FloatSlider(value=2.0, description='Fe2O3 (%):', max=20.0, min=1.0, style=SliderStyle(description_width='64pt'…

FloatSlider(value=1.0, description='MgO (%):', max=5.0, style=SliderStyle(description_width='64pt'))

...

FloatSlider(value=0.5, description='SO3 (%):', max=5.0, style=SliderStyle(description_width='64pt'))

FloatSlider(value=0.0, description='Humidity (%):', max=20.0, style=SliderStyle(description_width='64pt'))

FloatSlider(value=6.0, description='Clay Loss (%):', max=10.0, style=SliderStyle(description_width='64pt'))

Button(description='Run', style=ButtonStyle())

Output()

In [ ]:
# @title Strength of clinker
interact(strength,
         a=widgets.FloatSlider(value=55.0, min=45.0, max=65.0, step=0.1, description='C3S (%):'),
         b=widgets.FloatSlider(value=25.0, min=10.0, max=30.0, step=0.1, description='C2S (%):'),
         c=widgets.FloatSlider(value=10.0, min=5.0, max=12.0, step=0.1, description='C3A (%):'),
         d=widgets.FloatSlider(value=10.0, min=6.0, max=12.0, step=0.1, description='C4AF (%):'),
         e=widgets.FloatSlider(value=350.0, min=50.0, max=750.0, step=10, description='S. A. (m<sup>2</sup>/kg):'),
        );

interactive(children=(FloatSlider(value=55.0, description='C3S (%):', max=65.0, min=45.0), FloatSlider(value=2…

In [ ]:
# @title Bogue's equation to estimate composition from the clinker phases
interact(composition,
         a=widgets.FloatSlider(value=55.0, min=45.0, max=65.0, step=0.1, description='C3S (%):'),
         b=widgets.FloatSlider(value=25.0, min=10.0, max=30.0, step=0.1, description='C2S (%):'),
         c=widgets.FloatSlider(value=10.0, min=5.0, max=12.0, step=0.1, description='C3A (%):'),
         d=widgets.FloatSlider(value=10.0, min=6.0, max=12.0, step=0.1, description='C4AF (%):'),
        );

interactive(children=(FloatSlider(value=55.0, description='C3S (%):', max=65.0, min=45.0), FloatSlider(value=2…